### Para comenzar con el scrapping, importamos las funciones que tenemos en el archivo 'procesos'

In [5]:
from procesos import *

In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re

### Establecemos el nombre de las columnas que tendrá el dataframe

In [24]:
columnas = ['Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización']

### Llamamos a la función 'obtener_datos_obra' que realiza el proceso de web scraping (selenium), obtenemos los datos que queremos de la obra y lo convertimos en un dataframe

In [80]:
url = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/hidroelectrica-manuel-piar-tocoma/'
manuel_piar = obtener_datos_obra(url)

In [81]:
df = pd.DataFrame([manuel_piar]) #convertimos los datos en un dataframe

In [82]:
#Reorganizamos el orden de las columnas
df['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df = df[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df.columns = columnas
df

,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,Funcionario responsable,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización
0,HIDROELÉCTRICA MANUEL PIAR- TOCOMA,Paralizada,"87,19%",S/I,Estado Bolívar,Bolívar,"En el estado Bolívar, a 80 km. aguas arriba de...",Más de 1.000.001,Corporación Eléctrica Nacional (Corpoelec),Hipólito Izquierdo,Consorcio OIV Tocoma (CBPO Engenharia Ltda./ C...,USD 8.896.268.823,S/I,S/I,19/03/2007,S/I,1era 01/06/2014 2da 31/03/2018,S/I


### Vamos a realizar el mismo procedimiento para las demás obras que tenemos en nuestra página web que estamos scrapeando

In [83]:
url2 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/inversion-en-la-planta-india-urquia-perteneciente-al-complejo-generador-termocentro/'
india_urquia = obtener_datos_obra(url2)

In [84]:
df2 = pd.DataFrame([india_urquia])
df2.columns = columnas

In [85]:
url3 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/parque-eolico-de-la-guajira/'
parque_eolico = obtener_datos_obra(url3)

In [86]:
df3 = pd.DataFrame([parque_eolico])

df3 = df3.drop(df3.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df3['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df3['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df3 = df3[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido USD (millones)',14,16,16,17]]
df3.columns = columnas

In [87]:
url4 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/parque-eolico-paraguana/'
parque_eolico_paraguana = obtener_datos_obra(url4)

In [88]:
df4 = pd.DataFrame([parque_eolico_paraguana])

df4['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df4['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df4 = df4[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df4.columns = columnas

In [89]:
url5 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/proyecto-san-geronimo-cabruta-consolidacion-de-la-red-san-geronimo-puerto-ayacucho/'
geronimo_cabruta = obtener_datos_obra(url5)

In [90]:
df5 = pd.DataFrame([geronimo_cabruta])
df5.columns = columnas

In [91]:
url6 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/sistema-hidraulico-yacambu-quibor/'
yacambu_quibor = obtener_datos_obra(url6)

In [92]:
df6 = pd.DataFrame([yacambu_quibor])
df6.columns = columnas

### Procedemos a juntar todos los dataframes

In [93]:
# Concatena los DataFrames en uno solo
data_energia = pd.concat([df,df2,df3,df4,df5,df6],ignore_index=True)

### Para conocer las coordenadas de cada una de las obras, hacemos uso de Geopy

In [94]:
#Asignamos como dirección el Municipio y Estado de cada obra para conocer sus coordenadas
municipio_estado = data_energia['Municipio'] + ', ' + data_energia['Estado']
lista_municipio_estado = municipio_estado.tolist()

In [95]:
latitudes = obtener_latitudes(lista_municipio_estado) #obtenemos la latitud de la obra

In [96]:
longitudes = obtener_longitudes(lista_municipio_estado) #obtenemos la longitud de la obra

In [97]:
data_energia['Latitud'] = latitudes #agregamos una nueva columna con todos los valores de las latitudes
data_energia['Longitud'] = longitudes #agregamos una nueva columna con todos los valores de las longitudes

### Observamos que en las columnas 'Latitud' y 'Longitud' tenemos algunos valores nulos, esto se debe a que las obras abarcan mas de un estado, es por ello que procedemos a asignarles las coordenadas de los estados principales donde inician las obras

In [98]:
data_energia['Latitud'][4] = 9.9666667
data_energia['Longitud'][4] = --67.4666667

### Extraemos la parte numérica de la columna 'Valor total de la obra' para tener un dato de tipo entero

In [99]:
# Aplicar la función a la columna 'Valor total de la obra'
data_energia['Valor total de la obra USD (millones)'] = data_energia['Valor total de la obra USD (millones)'].apply(extract_numeric_value)

data_energia['Valor total de la obra USD (millones)'] = data_energia['Valor total de la obra USD (millones)'].str.replace('.', '', regex=True).fillna('0').astype(int)

#Dividimos para tener el total de millones en un número mas legible
data_energia['Valor total de la obra USD (millones)'] = (data_energia['Valor total de la obra USD (millones)'] / 1000000).astype(int)

### Limpiamos la columna 'Monto comprometido', extraemos la parte numérica, llenamos los valores nulos con '0', transformamos a un tipo de dato float 

In [100]:
data_energia['Monto comprometido USD (millones)'] = data_energia['Monto comprometido USD (millones)'].apply(extract_numeric_value)
data_energia['Monto comprometido USD (millones)'] = data_energia['Monto comprometido USD (millones)'].fillna('0')

# Convertir la columna a cadena, eliminar los puntos y reemplazar las comas por puntos
data_energia['Monto comprometido USD (millones)'] = data_energia['Monto comprometido USD (millones)'].astype(str)
data_energia['Monto comprometido USD (millones)'] = data_energia['Monto comprometido USD (millones)'].str.replace('.', '').str.replace(',', '.')

# Convertir la columna a tipo float
data_energia['Monto comprometido USD (millones)'] = pd.to_numeric(data_energia['Monto comprometido USD (millones)'], errors='coerce')

#Dividimos para tener el total de millones en un número más legible
data_energia['Monto comprometido USD (millones)'] = (data_energia['Monto comprometido USD (millones)'] / 1000000).astype(int)

### Cambiamos el tipo de dato de la columna 'Fecha estimada de inicio' por datetime y extraemos el año

In [101]:
# Convertir la columna 'Fecha estimada de inicio' a datetime
data_energia['Fecha estimada de inicio'] = pd.to_datetime(data_energia['Fecha estimada de inicio'], errors='coerce')
data_energia['Fecha estimada de inicio'] = data_energia['Fecha estimada de inicio'].dt.year

### Transformamos las columnas 'Ejecución física' y 'Ejecución financiera' en valores de tipo float, para ello llenamos los valores 'S/I' con 0 y extraemos la parte numérica

In [102]:
data_energia['Ejecución física %'] = data_energia['Ejecución física %'].apply(extract_numeric_value)
data_energia['Ejecución física %'] = data_energia['Ejecución física %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_energia['Ejecución física %'] = data_energia['Ejecución física %'].astype(float)

In [103]:
data_energia['Ejecución financiera %'] = data_energia['Ejecución financiera %'].apply(extract_numeric_value)
data_energia['Ejecución financiera %'] = data_energia['Ejecución financiera %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_energia['Ejecución financiera %'] = data_energia['Ejecución financiera %'].astype(float)

### Añadimos una nueva columna para indicar cuál es el sector correspondiente a la obra

In [104]:
data_energia['Sector'] = 'Energía'

In [106]:
#Reordenamos las columnas
nuevo_orden = ['Sector','Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización', 'Latitud', 'Longitud']
data_energia = data_energia[nuevo_orden]

In [107]:
data_energia

,Sector,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,...,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización,Latitud,Longitud
0,Energía,HIDROELÉCTRICA MANUEL PIAR- TOCOMA,Paralizada,87.19,0.0,Estado Bolívar,Bolívar,"En el estado Bolívar, a 80 km. aguas arriba de...",Más de 1.000.001,Corporación Eléctrica Nacional (Corpoelec),...,Consorcio OIV Tocoma (CBPO Engenharia Ltda./ C...,8896,S/I,0,2007.0,S/I,1era 01/06/2014 2da 31/03/2018,S/I,8.273929,-62.736852
1,Energía,INVERSIÓN EN LA PLANTA INDIA URQUÍA PERTENECIE...,Paralizada,80.00,0.0,Estado Miranda,Municipio Paz Castillo,"78Q9+RPM, Santa Lucía 1214",Más de 1.000.001,Corporación Eléctrica Nacional (Corpoelec) / M...,...,S/I,2178,Si,571,2009.0,S/I,2013,S/I,10.317713,-66.658116
2,Energía,PARQUE EÓLICO DE LA GUAJIRA,Paralizada,33.33,0.0,Estado Zulia,Guajira,"Sector Zulia Mar, parroquia Sinamaica. Municip...",Más de 1.000.001,Corporación Eléctrica Nacional (Corpoelec) / P...,...,Industrias Metalúrgicas Pescarmona Sociedad An...,225,S/I,0,NaN,2013,2013,2014,10.692780,-71.634994
3,Energía,PARQUE EÓLICO PARAGUANÁ,Paralizada,32.00,0.0,Estado Fálcon,Los Taques,"Santa Cruz de Los Taques, en la península de P...",Más de 1.000.001,Petróleos de Venezuela S.A. (Pdvsa) / Corporac...,...,S/I,180,S/I,0,2009.0,S/I,2011,S/I,11.824583,-70.251788
4,Energía,CONSOLIDACIÓN DE LA RED SAN GERÓNIMO-PUERTO AY...,Paralizada,8.76,0.0,Estado Guárico,Juan José Rondón/ Leonardo Infante,Municipios Juan José Rondón y Leonardo Infante,Más de 1.000.001,Corporación Eléctrica Nacional (Corpoelec) / M...,...,S/I,35,Si,170,2008.0,S/I,2010,S/I,9.966667,67.466667
5,Energía,SISTEMA HIDRÁULICO YACAMBÚ – QUIBOR,Paralizada,0.00,0.0,Estado Lara,Jiménez,"V8GX+57J, Carr. San José, 3061",Más de 1.000.001,Sistema Hidráulico Yacambú-Quíbor (SHYQ),...,Consorcio Yacambú 2008 /Empresa Noroccidental ...,879,Si,32,1999.0,S/I,2012,S/I,10.064215,-69.363809


### Exportamos nuestro dataframe

In [108]:
data_energia.to_csv('obras_energia.csv', index=False)